# <font color='blue'> **HW4 Dataset 2**</font>

In [35]:
#install.packages("MLmetrics")
#install.packages("gbm")
library(caret)
library(glmnet)
library(CVXR)
library(MLmetrics)
library(data.table)
library(glmnet)
library(gbm)
library(tidyverse)

### <font color='red'> **Dataset 2: Parkinson Speech Dataset with Multiple Types of Sound Recordings**</font>

Data Set Information:

The PD database consists of training and test files. The training data belongs to 20 PWP (6 female, 14 male) and 20 healthy individuals (10 female, 10 male) who appealed at the Department of Neurology in Cerrahpasa Faculty of Medicine, Istanbul University. From all subjects, multiple types of sound recordings (26 voice samples including sustained vowels, numbers, words and short sentences) are taken. A group of 26 linear and timeâ€“frequency based features are extracted from each voice sample. UPDRS ((Unified Parkinsonâ€™s Disease Rating Scale) score of each patient which is determined by expert physician is also available in this dataset. Therefore, this dataset can also be used for regression.

After collecting the training dataset which consists of multiple types of sound recordings and performing our experiments, in line with the obtained findings we continued collecting an independent test set from PWP via the same physicianâ€™s examination process under the same conditions. During the collection of this dataset, 28 PD patients are asked to say only the sustained vowels 'a' and 'o' three times respectively which makes a total of 168 recordings. The same 26 features are extracted from voice samples of this dataset. This dataset can be used as an independent test set to validate the results obtained on training set.

Further details are contained in the following reference -- if you use this dataset, please cite:
Erdogdu Sakar, B., Isenkul, M., Sakar, C.O., Sertbas, A., Gurgen, F., Delil, S., Apaydin, H., Kursun,
O., 'Collection and Analysis of a Parkinson Speech Dataset with Multiple Types of Sound
Recordings', IEEE Journal of Biomedical and Health Informatics, vol. 17(4), pp. 828-834, 2013

Training Data File:
Each subject has 26 voice samples including sustained vowels, numbers, words and short
sentences. The voice samples in the training data file are given in the
following order:

sample# - corresponding voice samples
1: sustained vowel (aaaâ€¦â€¦)
2: sustained vowel (oooâ€¦...)
3: sustained vowel (uuuâ€¦...)
4-13: numbers from 1 to 10
14-17: short sentences
18-26: words

Test Data File:
28 PD patients are asked to say only the sustained vowels 'a' and 'o' three times respectively which makes a total of 168 recordings (each subject has 6 voice samples) The voice samples in the test data file are given in the following order:

sample# - corresponding voice samples
1-3: sustained vowel (aaaâ€¦â€¦)
4-6: sustained vowel (oooâ€¦â€¦)


Attribute Information:

Training Data File:
column 1: Subject id

colum 2-27: features
features 1-5: Jitter (local),Jitter (local, absolute),Jitter (rap),Jitter (ppq5),Jitter (ddp),
features 6-11: Shimmer (local),Shimmer (local, dB),Shimmer (apq3),Shimmer (apq5), Shimmer (apq11),Shimmer (dda),
features 12-14: AC,NTH,HTN,
features 15-19: Median pitch,Mean pitch,Standard deviation,Minimum pitch,Maximum pitch,
features 20-23: Number of pulses,Number of periods,Mean period,Standard deviation of period, features 24-26: Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks

column 28: UPDRS
column 29: class information

Test Data File:
column 1: Subject id

colum 2-27: features
features 1-5: Jitter (local),Jitter (local, absolute),Jitter (rap),Jitter (ppq5),Jitter (ddp),
features 6-11: Shimmer (local),Shimmer (local, dB),Shimmer (apq3),Shimmer (apq5), Shimmer (apq11),Shimmer (dda),
features 12-14: AC,NTH,HTN,
features 15-19: Median pitch,Mean pitch,Standard deviation,Minimum pitch,Maximum pitch,
features 20-23: Number of pulses,Number of periods,Mean period,Standard deviation of period,
features 24-26: Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks

column 28: class information

In [2]:
fitControl = trainControl(method = "cv", number = 10)

In [22]:
glm_grid = expand.grid(alpha = 1, lambda = c(0,0.5,1))
dt_grid = expand.grid(cp = c(0.01,0.05,0.1))
rf_grid = expand.grid( mtry = c(3,5,7))
sgb_grid = expand.grid(n.trees = c(30, 50,100), shrinkage = c(0.05,0.1), interaction.depth = c(10, 20), n.minobsinnode = 10)

In [4]:
data1 = read.delim("train_data.txt", header = FALSE, sep = ",", dec = ".")
colnames(data1)[28]='Y'
data1 = data1[-c(1,29)]

In [5]:
head(data1)

V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,...,V19,V20,V21,V22,V23,V24,V25,V26,V27,Y
1.488,0.000090213,0.900,0.794,2.699,8.334,0.779,4.517,4.609,6.802,...,142.229,187.576,160,159,0.006064725,0.000416276,0.000,0,0.000,23
0.728,0.000037698,0.353,0.376,1.059,5.864,0.642,2.058,3.180,7.194,...,159.515,234.505,170,169,0.005181253,0.000403494,2.247,0,0.000,23
1.220,0.000074041,0.732,0.670,2.196,8.719,0.875,4.347,5.166,7.548,...,146.445,211.442,1431,1427,0.006070749,0.000474289,10.656,1,0.178,23
2.502,0.000122824,1.156,1.634,3.469,13.513,1.273,5.263,8.771,16.779,...,182.713,220.230,94,92,0.004910018,0.000319794,0.000,0,0.000,23
3.509,0.000166927,1.715,1.539,5.145,9.112,1.040,3.102,4.927,12.823,...,182.821,225.162,117,114,0.004757233,0.000379996,18.182,1,13.318,23
2.470,0.000126412,1.358,1.341,4.075,10.696,1.256,5.262,7.076,9.199,...,186.593,202.812,74,73,0.005117622,0.000187458,23.214,0,0.000,23


In [7]:
set.seed(1)
trainIndex = createDataPartition(data1$Y, p = 0.7, list = FALSE)
data1_train_1 = data1[trainIndex, ]
data1_test_1 = data1[-trainIndex, ]

In [10]:
set.seed(2)
data1_glm_model <- train(Y ~ ., data = data1_train_1, method = "glmnet", trControl = fitControl, tuneGrid = glm_grid)
data1_glm_model

glmnet 

728 samples
 26 predictor

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 655, 655, 654, 655, 655, 655, ... 
Resampling results across tuning parameters:

  lambda  RMSE      Rsquared    MAE     
  0.0     15.48447  0.09215025  12.39734
  0.5     15.43300  0.08877963  12.75596
  1.0     15.60268  0.06999723  13.14210

Tuning parameter 'alpha' was held constant at a value of 1
RMSE was used to select the optimal model using the smallest value.
The final values used for the model were alpha = 1 and lambda = 0.5.

In [13]:
data1_dt_model <- train(Y ~ ., data = data1_train_1, method = "rpart", trControl = fitControl, tuneGrid = dt_grid)
data1_dt_model

Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
"There were missing values in resampled performance measures."

CART 

728 samples
 26 predictor

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 656, 655, 655, 654, 656, 654, ... 
Resampling results across tuning parameters:

  cp     RMSE      Rsquared    MAE     
  0.005  17.07872  0.06997073  12.44186
  0.010  16.80469  0.05641592  12.31465
  0.050  16.01717  0.01621396  13.58714
  0.100  15.90069         NaN  13.61285

RMSE was used to select the optimal model using the smallest value.
The final value used for the model was cp = 0.1.

In [23]:
data1_rf_model = train(Y ~ ., data = data1_train_1, method = "rf", trControl = fitControl,tuneGrid = rf_grid)
data1_rf_model

Random Forest 

728 samples
 26 predictor

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 654, 656, 656, 655, 654, 655, ... 
Resampling results across tuning parameters:

  mtry  RMSE      Rsquared   MAE     
  3     14.64276  0.1676876  11.81817
  5     14.71322  0.1584345  11.82013
  7     14.72548  0.1544025  11.77763

RMSE was used to select the optimal model using the smallest value.
The final value used for the model was mtry = 3.

In [30]:
data1_sgb_model = train(Y ~ ., data = data1_train_1, method = "gbm", trControl = fitControl, tuneGrid = sgb_grid)
data1_sgb_model

Iter   TrainDeviance   ValidDeviance   StepSize   Improve
     1      247.8076             nan     0.0500    2.3322
     2      242.5115             nan     0.0500    2.7880
     3      237.2912             nan     0.0500    0.5400
     4      232.5317             nan     0.0500    2.8900
     5      228.0376             nan     0.0500    2.2992
     6      223.7868             nan     0.0500    0.7815
     7      219.7505             nan     0.0500    0.4179
     8      216.0311             nan     0.0500    1.0221
     9      212.6620             nan     0.0500    0.9737
    10      209.0612             nan     0.0500    1.7361
    20      182.5790             nan     0.0500    0.9393
    40      146.0457             nan     0.0500   -0.8449
    60      119.7403             nan     0.0500   -0.6658
    80      103.8910             nan     0.0500   -0.4322
   100       89.2175             nan     0.0500   -0.4763

Iter   TrainDeviance   ValidDeviance   StepSize   Improve
     1      2

Stochastic Gradient Boosting 

728 samples
 26 predictor

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 656, 656, 656, 655, 655, 655, ... 
Resampling results across tuning parameters:

  shrinkage  interaction.depth  n.trees  RMSE      Rsquared    MAE     
  0.05       10                  30      15.10231  0.10848558  12.25250
  0.05       10                  50      15.08029  0.11346872  11.96672
  0.05       10                 100      15.26090  0.11005790  11.85239
  0.05       20                  30      15.05245  0.11514832  12.06669
  0.05       20                  50      15.09808  0.11590165  11.81214
  0.05       20                 100      15.34409  0.10979589  11.78345
  0.10       10                  30      15.32729  0.09998169  11.98748
  0.10       10                  50      15.40616  0.10746538  11.89946
  0.10       10                 100      15.64327  0.10694626  11.95729
  0.10       20                  30      15.44079  0.099690

In [31]:
data1_glm_model$bestTune
data1_dt_model$bestTune
data1_rf_model$bestTune
data1_sgb_model$bestTune

,alpha,lambda
2,1,0.5


,cp
4,0.1


mtry
3


,n.trees,interaction.depth,shrinkage,n.minobsinnode
4,30,20,0.05,10


The best parameters for 4 models are shown above.

In [36]:
glm_RMSE = min(data1_glm_model$result$RMSE)
dt_RMSE = min(data1_dt_model$result$RMSE)
rf_RMSE = min(data1_rf_model$result$RMSE)
sgb_RMSE = min(data1_sgb_model$result$RMSE)
data.table(glm_RMSE,dt_RMSE,rf_RMSE,sgb_RMSE)

glm_RMSE,dt_RMSE,rf_RMSE,sgb_RMSE
15.433,15.90069,14.64276,15.05245


In [37]:
glm_MAE = min(data1_glm_model$result$MAE)
dt_MAE = min(data1_dt_model$result$MAE)
rf_MAE = min(data1_rf_model$result$MAE)
sgb_MAE = min(data1_sgb_model$result$MAE)
data.table(glm_MAE,dt_MAE,rf_MAE,sgb_MAE)

glm_MAE,dt_MAE,rf_MAE,sgb_MAE
12.39734,12.31465,11.77763,11.78345


As seen the results above, rf model gives the best result.

In [38]:
data1_glm_predict <- predict(data1_glm_model,data1_test_1)   
data1_dt_predict <- predict(data1_dt_model,data1_test_1)   
data1_rf_predict <- predict(data1_rf_model,data1_test_1)   
data1_sgb_predict <- predict(data1_sgb_model,data1_test_1)   

In [40]:
glm_Test_RMSE = RMSE(data1_test_1$Y,data1_glm_predict)
dt_Test_RMSE = RMSE(data1_test_1$Y,data1_dt_predict)
rf_Test_RMSE = RMSE(data1_test_1$Y,data1_rf_predict)
sgb_Test_RMSE = RMSE(data1_test_1$Y,data1_sgb_predict)
RSME_result = data.table(glm_Test_RMSE,dt_Test_RMSE,rf_Test_RMSE,sgb_Test_RMSE)
RSME_result

glm_Test_RMSE,dt_Test_RMSE,rf_Test_RMSE,sgb_Test_RMSE
15.03932,15.78541,14.1384,14.50113


As same as train dataset, RMSE value is the best for RF model. Therefore RF Model with MTRY=3 is the best for this dataset.